In [1]:
"""Test the radiosity.Radiosity module."""
import numpy as np
import pyfar as pf
import sparrowpy as sp

# from sparrowpy import image_source as ims
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
%matplotlib widget

# ruff: noqa: ERA001

Define the parameters and geometry for the simulation.

In [2]:
# room dimensions
import math

# R = 1.5
planeZ = 5  # height of the plane in space
height = 1  # height of the source
min_XY = 0.01  # plane dimensions in meters
max_XY = 100

check_patch_size = 0  # meant as check_plane_dimensions = not check_patch_size
resolution_of_sim = 40  # resolution of the range of simulated geometry
res = max(10, resolution_of_sim)  # min sim res is 10
patch_max_size_structure_inf_plane = 0.3  # cutoff for patch size for large pl
patch_max_size = 2
patch_min_size = 0.05
ir_length_s = 1  # min length otherwise out of bounds refl for first order
sampling_rate = 1000
max_order_k = 1
speed_of_sound = 346.18
absorption = 1  # param default 0.1

# create geometry
# for logspace logspace(-2, 6.64, res, base=2) # length for 0.25 to 100m
min_XY = math.log2(min_XY)
max_XY = math.log2(max_XY)

# circleRes = np.arange(0, 2 * np.pi * R, 0.1)  # circle of 1.5 meters
# probeDims = np.column_stack(
#    (np.cos(circleRes), np.sin(circleRes), [5] * len(circleRes)),
# )
debug__size_plane = np.logspace(min_XY, max_XY, res, base=2)
index_of_8m = (np.abs(debug__size_plane - 8)).argmin()  # 0.25m-100m res30 ~8m [17]

elements = [  # [17/index_of_8m] corner 8.37m
    sp.geometry.Polygon(
        np.array(
            [
                [0, 0, planeZ],
                [0, i, planeZ],
                [i, i, planeZ],
                [i, 0, planeZ],
            ],
        ),
        np.array([1, 0, 0]),
        np.array([0, 0, 1]),
    )
    for i in np.logspace(min_XY, max_XY, res, base=2)  # length for 0.25 to 100m
]

source_pos = [
    np.array([i / 2, i / 2, planeZ + height]) for i in np.logspace(min_XY, max_XY, res, base=2)
]
sources = [
    sp.geometry.SoundSource(spos, np.array([0, 1, 0]), np.array([0, 0, 1]))
    for spos in source_pos
]
receiver_pos = source_pos  # sub to change

Calculate old slow approach for comparison.

In [3]:
from numpy import ndarray


start = datetime.now()

patch_values = np.arange(
    patch_max_size,
    patch_min_size,
    -(patch_max_size - patch_min_size) / res,
)

if check_patch_size:
    radiPlanes = [
        sp.DRadiosityFast.from_polygon(
            [elements[index_of_8m]],
            i,
        )
        for i in patch_values
    ]
else:
    radiPlanes = [
        sp.DRadiosityFast.from_polygon(
            [element],  # singular element
            (float)(
                np.min(
                    [
                        (np.logspace(min_XY, max_XY, res, base=2)[index] ** 2 / 2000) ** 0.5,
                        patch_max_size_structure_inf_plane,
                    ],
                ),
            ),
        )
        for index, element in enumerate(elements)
    ]

# values for simulation
source_ = pf.samplings.sph_gaussian(sh_order=5)
source_ = source_[source_.z >= 0]
receiver_ = pf.samplings.sph_gaussian(sh_order=5)
receiver_ = receiver_[receiver_.z >= 0]
# print(f"number of sources {source_.csize} and receivers {receiver_.csize}")
frequencies = np.array([500])
data_scattering = sp.brdf.create_from_scattering(
    source_, receiver_, pf.FrequencyData(np.zeros_like(frequencies), frequencies)
)
brdf_sources = pf.Coordinates(0, 0, 1, weights=1)
brdf_receivers = pf.Coordinates(0, 0, 1, weights=1)
brdf = sp.brdf.create_from_scattering(
    brdf_sources,
    brdf_receivers,
    pf.FrequencyData(1, [100]),
    pf.FrequencyData(0, [100]),
)


# Initialize histograms
histograms = np.zeros_like(radiPlanes)
for indexForRadiPlanes, sim in enumerate(radiPlanes):
    sim.set_wall_scattering([0], brdf, brdf_sources, brdf_receivers)  # len(elements) eq 0

    sim.set_air_attenuation(
        pf.FrequencyData(np.zeros_like(brdf.frequencies), brdf.frequencies)
    )

    sim.set_wall_absorption(
        [0],  # len(elements) eq 0
        pf.FrequencyData(
            np.zeros_like(brdf.frequencies),
            brdf.frequencies,
        ),
    )

    # calculate from factors including directivity and absorption not need bc only one plane
    # sim.bake_geometry()

    if check_patch_size:
        sim.init_source_energy(
            source_pos[index_of_8m],
        )
    else:
        sim.init_source_energy(
            source_pos[indexForRadiPlanes],  # source_pos[indexForRadiPlanes]
        )

    sim.calculate_energy_exchange(
        # receiver_pos[dependent on relevant index #1 or #radiPlanes], sim knows receiver_pos
        speed_of_sound=speed_of_sound,
        histogram_time_resolution=1 / sampling_rate,
        histogram_length=ir_length_s,
        max_depth=max_order_k,
    )

    # gather energy at receiver  # receiver not as geometry #i.energy_at_receiver
    if check_patch_size:
        histograms += sim.collect_receiver_energy(
            receiver_pos=receiver_pos[index_of_8m],
            speed_of_sound=speed_of_sound,
            histogram_time_resolution=1 / sampling_rate,
        )
    else:
        histograms += sim.collect_receiver_energy(
            receiver_pos=receiver_pos[indexForRadiPlanes],
            speed_of_sound=speed_of_sound,
            histogram_time_resolution=1 / sampling_rate,
        )


# stop timer
delta = datetime.now() - start
print(f"Time elapsed: {delta}")
slow_time_s = (delta.seconds * 1e6 + delta.microseconds) / 1e6

C:\Users\klingelnberg\AppData\Local\Temp\ipykernel_11696\1479302272.py:37: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.gaussian.'
  source_ = pf.samplings.sph_gaussian(sh_order=5)
c:\Users\klingelnberg\AppData\Local\miniconda3\envs\sparrowpy\Lib\site-packages\pyfar\samplings\samplings.py:344: PyfarDeprecationWarning: 'This function will be changed in pyfar 0.8.0 to init(x, y, z).'
  sampling = pyfar.Coordinates(
c:\Users\klingelnberg\AppData\Local\miniconda3\envs\sparrowpy\Lib\site-packages\pyfar\classes\coordinates.py:265: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.SamplingSphere.'
  self.sh_order = sh_order
C:\Users\klingelnberg\AppData\Local\Temp\ipykernel_11696\1479302272.py:39: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.gaussian.'
  receiver_ = pf.samplings.sph_gaussian(sh_order=5)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(int64, Literal[int](0))
 
There are 22 candidate implementations:
[1m   - Of which 22 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(int64, int64)':[0m
[1m    No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\klingelnberg\BA\sparrowpy\sparrowpy\radiosity_fast\geometry.py (124)[0m
[0m[1mDuring: typing of static-get-item at C:\Users\klingelnberg\BA\sparrowpy\sparrowpy\radiosity_fast\geometry.py (124)[0m
[1m
File "..\sparrowpy\radiosity_fast\geometry.py", line 124:[0m
[1mdef get_scattering_data_source(
    <source elided>
    source_idx = np.argmin(np.sum(
[1m        (sources[wall_id_i[0], :, :]-difference_source)**2, axis=-1))


Diffuse and specular Intensity with plotting

In [ ]:
# Signal
I_diffuse_planes = [
    pf.Signal(i, sampling_rate=sampling_rate) for i in histograms
]

I_specular = 1 / (4 * np.pi * (height * 2) ** 2)
factors = [np.sum(i.time[:, :]) / I_specular for i in I_diffuse_planes]

# Plot the factors
plt.figure()
if check_patch_size:
    plt.plot(patch_values, factors, label="factor")
    plt.gca().invert_xaxis()
    plt.xlabel("patch size")
else:
    plt.plot(np.logspace(min_XY, max_XY, res, base=2), factors, label="factor")
    plt.xlabel("plane width/length")
    plt.xscale("log")

    plt.figure()
    plt.plot(
        np.arctan(np.logspace(min_XY, max_XY, res, base=2) / 2 / height)
        / np.pi
        * 180,
        [(i / 2) for i in factors],
        label="factor",
    )
    plt.xlabel("plane edge angle to mid positioned source=receiver")
    plt.xscale("linear")

plt.ylabel("I_diffuse / I_specular")
plt.legend()
plt.show()

Find signal points

In [ ]:
%load_ext watermark
%watermark -v -m -iv